<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/chuongmep/aps-toolkit/blob/dev/APSToolkitPython/Tutorials/04.01.%20Explore%20Revit%20Data%20With%20APSToolkit.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/chuongmep/aps-toolkit/blob/dev/APSToolkitPython/Tutorials/04.01.%20Explore%20Revit%20Data%20With%20APSToolkit.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<table align="left">
  <tr>
    <td>Author(s)</td>
    <td><a href="https://github.com/chuongmep" id="author-link">Chuong Ho</a></td>
    <!-- <td><a href="<your repo>" id="github-link">YourName</a></td> -->
  </tr>
</table>

In [1]:
%pip install aps-toolkit --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: aps-toolkit
    Found existing installation: aps-toolkit 0.6.7
    Uninstalling aps-toolkit-0.6.7:
      Successfully uninstalled aps-toolkit-0.6.7
Note: you may need to restart the kernel to use updated packages.


In [2]:
from aps_toolkit import Auth
import pandas as pd

### Authentication

In [3]:
token = Auth().auth2leg()

#### Auth Via Google Colab

In [ ]:
# from google.colab import userdata
# client_id = userdata.get('APS_CLIENT_ID')
# client_secret = userdata.get('APS_CLIENT_SECRET')
# auth = Auth(client_id, client_secret)
# token = auth.auth2leg()

In [4]:
urn = "dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLk9kOHR4RGJLU1NlbFRvVmcxb2MxVkE_dmVyc2lvbj0yNA"

### Reader

#### General Reader with Default View

In [5]:
from aps_toolkit import PropDbReaderRevit

prop_reader = PropDbReaderRevit(urn, token)

#### Reader with Custom View

In [6]:
from aps_toolkit import Derivative
derivative = Derivative(urn, token,"US")

In [7]:
items = derivative.read_svf_manifest_items()
for item in items:
    print(item.path_info.root_file_name)

MyRoom.svf
Chair.svf
Bed.svf
3D_MyHouse.svf
{3D - Chuong_Ho}.svf
3D_Data Xchange.svf
{3D - vanchuong_ho}.svf
Project Completion.svf


In [8]:
item = items[0]
print("Guid: ", item.guid)
print("Root File Name Item: ", item.path_info.root_file_name)
print("Path: ", item.path_info.files)

Guid:  a4e4238d-a839-5b6a-ad55-42f5e125f820
Root File Name Item:  MyRoom.svf
Path:  ['../../objects_attrs.json.gz', '../../objects_vals.json.gz', '../../objects_ids.json.gz', '../../objects_viewables.json.gz', '../../objects_offs.json.gz', '../../objects_avs.json.gz', 'Materials.json.gz', 'CameraDefinitions.bin', 'LightDefinitions.bin', '0.pf', '1.pf', '2.pf', '3.pf', '4.pf', '5.pf', 'GeometryMetadata.pf', 'FragmentList.pack', 'CameraList.bin', 'LightList.bin', 'InstanceTree.bin', 'Set.bin', 'MyRoom.svf', 'MyRoom1.png', 'MyRoom2.png', 'MyRoom4.png']


In [9]:
prop_reader = PropDbReaderRevit(urn, token,"US",items[0])

In [20]:
df = prop_reader.get_data_by_categories_and_params(["Doors", "Windows"], ["Width","Category", "Height", "Length", "Workset"], True)
pd.set_option('display.max_columns', 5)
pd.set_option('display.width', 1000)
df

,dbId,external_id,...,Height,Width
0,3551,5bb069ca-e4fe-4e63-be31-f8ac44e80d30-000471ee,...,2133.6,914.400
1,3902,6d22740f-4d3f-4cc6-a442-8c98ddd54f1f-0004923b,...,1524.0,1219.200
2,3907,6d22740f-4d3f-4cc6-a442-8c98ddd54f1f-000492aa,...,1384.3,473.075
3,3908,6d22740f-4d3f-4cc6-a442-8c98ddd54f1f-000492ab,...,1384.3,473.075
4,3912,6d22740f-4d3f-4cc6-a442-8c98ddd54f1f-000492ac,...,NaN,NaN
5,3916,6d22740f-4d3f-4cc6-a442-8c98ddd54f1f-000492ad,...,NaN,NaN


In [21]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [22]:
agent = create_pandas_dataframe_agent(ChatOpenAI(temperature=0), df, verbose=True)

In [23]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [24]:
agent.invoke("how many category there? what kind of categories?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Category'].nunique()"}`


3
Invoking: `python_repl_ast` with `{'query': "df['Category'].unique()"}`


['Revit Doors' 'Revit Windows' 'Revit Generic Models']There are 3 categories in the dataframe. The categories are 'Revit Doors', 'Revit Windows', and 'Revit Generic Models'.

> Finished chain.


{'input': 'how many category there? what kind of categories?',
 'output': "There are 3 categories in the dataframe. The categories are 'Revit Doors', 'Revit Windows', and 'Revit Generic Models'."}